In [1]:
from qiskit import *
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
import itertools
import numpy as np
from time import sleep
import pandas as pd
import progressbar
import csv
sim = Aer.get_backend('qasm_simulator')
%matplotlib inline

c:\Users\taren\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\taren\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\taren\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Find all permutations and synonyms

In [2]:
gates = ['x', 'y', 'z', 'h']
gateCombos = []
comboSV = []
similar = []

In [3]:
#Find all permutations and append their statevectors and permutations to lists
for L in range(1, len(gates)+1):
    
    #for subset in itertools.permutations(gates, L):
    for subset in itertools.product(gates, repeat=L):
        #print(subset)
        gateCombos.append(subset)
#Add a no gate option
gateCombos.append('')
        
#find statevectors of each possible permutation and store in list
for i in range(len(gateCombos)):
    qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
    #print(gateCombos[i])
    
    for j in gateCombos[i]:
        getattr(qc , j)(0)
        #print(qc)
    # Let's get the result:
    qc.save_statevector()
    qobj = assemble(qc)
    result = sim.run(qobj).result()
    # Print the statevector neatly:
    final_state = result.get_statevector()
    array_to_latex(final_state, prefix="\\text{Statevector = }")
    comboSV.append(final_state)


In [4]:
len(gateCombos)
print(gateCombos[len(gateCombos)-1])

In [5]:
len(comboSV)
print(comboSV[len(comboSV)-1])

Statevector([1.+0.j, 0.+0.j],
            dims=(2,))


In [6]:
#Create a list of synonym pairs based on if statevectors are equal
bar = progressbar.ProgressBar(maxval=len(comboSV)).start()
for i in range(len(comboSV)):
    for j in range(len(comboSV)):
        temp = []
        if(i != j and comboSV[i]==comboSV[j]):
            temp.append(gateCombos[i])
            temp.append(gateCombos[j])
            #print(temp)
            similar.append(temp)
    bar.update(i)
            

In [7]:
len(similar)

8330

In [8]:
bar = progressbar.ProgressBar(maxval=len(similar)).start()
# provider = IBMQ.get_provider('ibm-q')
# backend = provider.get_backend('ibmq_armonk')
#sort largest synonym in pair first
#If synonyms are the same in length, check computing times of each and set slower one as target and quicker one as the replacement
listToRemove = []
for i in range(len(similar)):
    if len(similar[i][1]) > len(similar[i][0]):
        similar[i].insert(0, similar[i].pop(1))
    elif len(similar[i][1]) == len(similar[i][0]):
        #1
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for j in similar[i][0]:
            getattr(qc , j)(0)
        qc.measure(0, 0)
        result = execute(qc, backend=sim, shots = 1000).result()
        time1 = result.time_taken
        #2
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for j in similar[i][1]:
            getattr(qc , j)(0)
        qc.measure(0, 0)
        result = execute(qc, backend=sim, shots = 1000).result()
        time2 = result.time_taken
        if time2 > time1:
            similar[i].insert(0, similar[i].pop(1))
        if time2 == time1:
            listToRemove.append(i)
    bar.update(i)
listToRemove.sort(reverse=True)
print("Removing Elements at Indices: " + str(listToRemove))
listToRemove.sort(reverse=True)
for r in listToRemove:
    similar.pop(r)

Removing Elements at Indices: [8273, 8154, 8100, 8069, 8025, 8013, 8002, 7943, 7906, 7880, 7874, 7846, 7810, 7805, 7764, 7705, 7690, 7623, 7620, 7609, 7602, 7557, 7540, 7511, 7495, 7491, 7470, 7458, 7457, 7423, 7338, 7333, 7331, 7283, 7242, 7118, 7108, 7096, 7095, 7082, 7049, 6941, 6926, 6898, 6847, 6846, 6844, 6753, 6704, 6639, 6630, 6614, 6569, 6565, 6505, 6502, 6448, 6387, 6370, 6310, 6309, 6263, 6226, 6196, 6154, 6143, 6141, 6134, 6075, 5964, 5871, 5844, 5836, 5730, 5685, 5641, 5601, 5510, 5478, 5428, 5253, 5249, 5211, 5168, 5096, 5064, 5058, 5020, 4994, 4979, 4938, 4911, 4907, 4899, 4852, 4830, 4816, 4738, 4731, 4668, 4661, 4633, 4625, 4547, 4509, 4450, 4448, 4433, 4411, 4378, 4292, 4272, 4236, 4168, 4143, 4132, 4061, 4049, 4028, 3981, 3974, 3965, 3942, 3935, 3903, 3902, 3869, 3829, 3815, 3760, 3748, 3743, 3716, 3710, 3706, 3617, 3601, 3597, 3590, 3575, 3411, 3390, 3384, 3376, 3374, 3363, 3360, 3358, 3348, 3342, 3323, 3320, 3280, 3238, 3223, 3211, 3210, 3203, 3195, 3126, 3121, 310

In [9]:
len(similar)

8146

## Create Database File

In [10]:
with open(r"Synonym_Database\key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(similar)



### CSV to Pandas DF

In [11]:
df = pd.read_csv(r"Synonym_Database\key.csv")

### Filtering out multiple replacements for one target

In [12]:
unique = df['Target'].unique()
unique = unique.tolist()
targets = df['Target'].tolist()

In [13]:
bar = progressbar.ProgressBar(maxval=len(unique)).start()
remove = []
for i in range(len(unique)):
    bar.update(i)
    times = []
    indices = []
    for j in range(len(targets)):
        if unique[i] == targets[j]:
            indices.append(j)
    for x in range(len(indices)):
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for q in similar[x][1]:
            getattr(qc , q)(0)
        qc.measure(0, 0)
        result = execute(qc, backend=sim, shots = 1000).result()
        times.append(result.time_taken)
    least = min(times)
    for e in indices:
        remove.append(e)
    remove.remove(indices[times.index(least)])
remove.sort(reverse=True)
#print(remove)
for i in remove:
    similar.pop(i)

In [14]:
len(similar)

328

In [15]:
with open(r"Synonym_Database\key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(similar)

## Reading and Cleaning File

In [16]:
#Get simplist replacements for each target
keyDF = pd.read_csv(r"Synonym_Database\key.csv")

for item in range(len(keyDF)):
    for m in range(0, 50):
        for element in range(len(keyDF)):
            if keyDF['Target'][element] == keyDF['Replacement'][item]:
                # print("1. " + str(keyDF['Replacement'][item]))
                # print("2. " + str(keyDF['Target'][element]))
                keyDF['Replacement'][item] = keyDF['Replacement'][element]

lst = []
print(keyDF)
for i in range(len(keyDF)):
    temp = []
    temp.append(keyDF['Target'][i])
    temp.append(keyDF['Replacement'][i])
    lst.append(temp)
with open(r"Synonym_Database\key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(lst)

                   Target Replacement
0         ('y', 'x', 'x')      ('y',)
1         ('y', 'y', 'y')      ('y',)
2    ('z', 'y', 'y', 'y')      ('y',)
3    ('h', 'y', 'h', 'z')      ('y',)
4         ('z', 'x', 'x')         NaN
..                    ...         ...
323  ('h', 'h', 'h', 'x')      ('h',)
324  ('h', 'h', 'h', 'y')  ('h', 'y')
325  ('h', 'z', 'y', 'y')  ('x', 'h')
326  ('h', 'h', 'h', 'h')         NaN
327            ('x', 'x')         NaN

[328 rows x 2 columns]
